# Assessment 3: WebCrawler and NLP System 
# Lynne Parkinson

# TASK 2 - WebCrawler

# a. ABC SELENIUM URL List Scraping

In [ ]:
#https://www.geeksforgeeks.org/get-all-text-of-the-page-using-selenium-in-python/
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

chromeOptions = Options()
chrome_prefs = {"profile.managed_default_content_settings.images": 2}
chromeOptions.add_experimental_option("prefs", chrome_prefs)
chromeOptions.add_argument("--disable-infobars")
chromeOptions.add_argument("--disable-extensions")
chromeOptions.add_argument("--ignore-certificate-errors")
chromeOptions.add_argument("--incognito")
chromeOptions.add_argument('--headless')
chromeOptions.add_argument("--no-sandbox")
chromeOptions.add_argument('disable-notifications')
chrome_prefs = {"profile.managed_default_content_settings.images": 2}
chromeOptions.add_experimental_option("prefs", chrome_prefs)

driver = webdriver.Chrome(options=chromeOptions)

with open("ABC_Links.csv") as in_file:
    for url in in_file:
        # get website
        driver.get(url.strip())
        # get current url
        print(driver.current_url)
        # Storing the page source in page variable 
        page = driver.page_source.encode('utf-8') #https://www.geeksforgeeks.org/get-contents-of-entire-page-using-selenium/
        file_ = open('result.html', 'ab')
        # Write the entire page content in result.html 
        file_.write(page) 
        time.sleep(5)
  
# Closing the file 
file_.close() 
print('end of scrape')
  
# Closing the driver 
driver.quit()

# b. ABC BS4 parsing

In [1]:
# import required libraries and packages
from bs4 import BeautifulSoup
import pandas as pd


In [2]:
#https://oxylabs.io/blog/beautiful-soup-parsing-tutorial
#finding html tags
with open('result.html', 'r', encoding="utf8") as f:
    contents = f.read()

    soup = BeautifulSoup(contents, "html.parser")

    for child in soup.descendants:

       if child.name:
           print(child.name)

In [11]:
#Find all instances of title, extract text and save to txt.file
title_list = soup.find_all('h1')
with open('title.txt', 'w', encoding='utf-8') as f:
    for title in title_list:
        f.write(f'{title.text}\n')
        print(title_list)

In [10]:
#Find all instances of p, extract text and save to txt.file
body_list = soup.find_all('p')
with open('body.txt', 'w', encoding='utf-8') as f:
    for body in body_list:
        f.write(f'{body.text}\n')

In [5]:
#Find all instances of time, extract text and save to file 
tim_list = soup.find_all('time')
with open('tim.txt', 'w', encoding='utf-8') as f:
    for tim in tim_list:
        f.write(f'{tim.text}\n')
 

#  Task 3 Data Wrangling and Exploration of Corpus

# a. Explore Sample (all documents, not cleaned)

In [ ]:
# Text exploratory analysis
import re
from nltk import FreqDist
# Visualisation
import matplotlib.pyplot as plt


In [ ]:
sample =  pd.read_fwf('body2.txt', sep="", header=None)
sample.head()

In [ ]:
#Remove NANS
sample=sample.dropna()

In [ ]:
# Save sample to csv
sample.to_csv('body.csv')

In [ ]:
# PLOTTING WORD-COUNT
sample['word_count'] = sample[0].apply(lambda x: len(str(x).split()))
fig=plt.hist(sample['word_count'])
plt.hist(sample['word_count'],color='blue', label=None)
plt.title('Article Word Count')
plt.show()

In [ ]:
#Prepare body into one giant string
big_string = " ".join(sample[0].values)
print(f"***** Extract of sample big_string ***** \n{big_string[:101]}", "\n")
# Split sample by white space
splits = big_string.split()  
print(f"***** Extract of sample splits ***** \n{splits[:18]}\n")


In [ ]:
# Number of strings
print(f"Number of strings: {len(splits)}")
print(f"Number of unique strings: {len(set(splits))}")

In [ ]:
# Most frequent splits
freq_splits = FreqDist(splits)
print(f"***** 10 most common strings ***** \n{freq_splits.most_common(10)}", "\n")

In [ ]:
# Short splits
short = set(s for s in splits if len(s)<4)
short = [(s, freq_splits[s]) for s in short]
short.sort(key=lambda x:x[1], reverse=True)
short

In [ ]:
# Long splits
long = set(s for s in splits if len(s)>10)
long = [(s, freq_splits[s]) for s in long]
long.sort(key=lambda x:x[1], reverse=True)
long

In [ ]:
# Summarise patterns and strings
def summarise(pattern, strings, freq):
    """Summarise strings matching a pattern."""
    # Find matches
    compiled_pattern = re.compile(pattern)
    matches = [s for s in strings if compiled_pattern.search(s)]
    
    # Print volume and proportion of matches
    print("{} strings, that is {:.2%} of total".format(len(matches), len(matches)/ len(strings)))
    
    # Create list of tuples containing matches and their frequency
    output = [(s, freq[s]) for s in set(matches)]
    output.sort(key=lambda x:x[1], reverse=True)
    
    return output

In [ ]:
# Find strings possibly containing html tag
summarise(r"/?>?w*<|/>", splits, freq_splits)

In [ ]:
# Find strings possibly containing numbers
summarise(r"\d", splits, freq_splits)

# b. Data Wrangling

In [ ]:
# Import needed packages
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt') # for sent_tokenize
nltk.download('wordnet') # for WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.stem.snowball import SnowballStemmer
st = SnowballStemmer('english')

# Data manipulation/analysis
import numpy as np

# Text preprocessing/analysis
import re
from nltk import word_tokenize, sent_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid", context='talk', 
        palette=['#D44D5C', '#43AA8B'])


In [ ]:
# Open Parsed Data
# convert csv file to data frame
data_df = pd.read_csv("body2.txt", sep='\t',
                         names=['Body', 'Body2', 'Body3'])
 # printing data frame
print("Data frame")
print(data_df)
 
# printing row header
print("Row header")
print(list(data_df.columns))

In [ ]:
# Choose 'body' only
data=data_df[['Body']]
data.head()

In [ ]:
# Function to clean data
def clean_data(df, col, clean_col):

    # change to lower and remove spaces on either side
    df[clean_col] = df[col].apply(lambda x: x.lower().strip())

    # remove extra spaces in between
    df[clean_col] = df[clean_col].apply(lambda x: re.sub(' +', ' ', x))

    # remove punctuation
    df[clean_col] = df[clean_col].apply(lambda x: re.sub('[^a-zA-Z]', ' ', x))

    # remove stopwords and get the stem
    df[clean_col] = df[clean_col].apply(lambda x: ' '.join(st.stem(text) for text in x.split() if text not in stop_words))

    return df

In [ ]:
# Calling cleaning function
corpus = clean_data(data, 'Body', 'clean_body')

In [ ]:
corpus=corpus[['clean_body']]
corpus.head()

In [ ]:
corpus.to_csv('clean_body.csv', index=None)
corpus.to_csv('clean_body_2.csv')

# c. Explore Corpus

In [ ]:
# Define Word Count
count = corpus['clean_body'].str.split().str.len()
count.index = count.index.astype(str) + ' words:'
count.sort_index(inplace=True)
count
df=count

In [ ]:
# Plot Corpus word length
plt.figure(figsize=(12, 12))
sns.countplot(y=count)
plt.title("Corpus Text Length Distribution", size=20);

In [ ]:
# Prepare corpus into one giant string
print(f"***** Extract of corpus string ***** \n{corpus[:101]}", "\n")

In [ ]:
# Split corpus by white space
corpus = corpus.to_string() 
splits = corpus.split()  
print(f"***** Extract of corpus splits ***** \n{splits[:18]}\n")

In [ ]:
# Number of strings
print(f"Number of strings: {len(splits)}")
print(f"Number of unique strings: {len(set(splits))}")

In [ ]:
# Most frequent splits
freq_splits = FreqDist(splits)
print(f"***** 10 most common strings ***** \n{freq_splits.most_common(10)}", "\n")

In [ ]:
# Short splits
short = set(s for s in splits if len(s)<4)
short = [(s, freq_splits[s]) for s in short]
short.sort(key=lambda x:x[1], reverse=True)
short

In [ ]:
# Long splits
long = set(s for s in splits if len(s)>10)
long = [(s, freq_splits[s]) for s in long]
long.sort(key=lambda x:x[1], reverse=True)
long

In [ ]:
# Summarise patterns and strings
def summarise(pattern, strings, freq):
    
        # Find matches
    compiled_pattern = re.compile(pattern)
    matches = [s for s in strings if compiled_pattern.search(s)]
    
    # Print volume and proportion of matches
    print("{} strings, that is {:.2%} of total".format(len(matches), len(matches)/ len(strings)))
    
    # Create list of tuples containing matches and their frequency
    output = [(s, freq[s]) for s in set(matches)]
    output.sort(key=lambda x:x[1], reverse=True)
    
    return output

In [ ]:
# Find strings possibly containing html tag
summarise(r"/?>?w*<|/>", splits, freq_splits)

In [ ]:
# Find strings possibly containing html tag
summarise(r"\d", splits, freq_splits)

In [ ]:
# What are the most frequent stop words?
stop_words = stopwords.words("english")
print(f"There are {len(stop_words)} stopwords.\n")
print(stop_words)

In [ ]:
# extend stopwords
stop_words.extend(["cannot", "could", "done", "let", "may" "mayn",  "might", "must", "need", "ought", "oughtn", "shall", "would", "br"])
print(f"There are {len(stop_words)} stopwords.\n")

In [ ]:
# Frequency of stopwords in corpus
freq_stopwords = [(sw, tokens_norm.count(sw)) for sw in stop_words]
freq_stopwords.sort(key=lambda x: x[1], reverse=True)
freq_stopwords[:192]

In [ ]:
# What is the average number of characters per token?
# Create list of token lengths for each token
token_length = [len(t) for t in tokens]
# Average number of characters per token
print(f"Average number of characters per token: {round(np.mean(token_length),4)}")
# Plot distribution
plt.figure(figsize=(12, 12))
sns.countplot(y=token_length)
plt.title("Counts of token length", size=20);

# d. Prepare corpus for ML approach

In [ ]:
# Tokenise
# tokeniser = RegexpTokenizer("[A-Za-z]+")
tokens = tokeniser.tokenize(corpus)
with open("tokens.txt", "w") as output:
    output.write(str(tokens))
print(tokens[:20], "\n")

In [ ]:
print(f"Number of tokens: {len(tokens)}")
print(f"Number of unique tokens: {len(set(tokens))}")

In [ ]:
# Leammatise
lemmatiser = WordNetLemmatizer()
tokens_norm = [lemmatiser.lemmatize(t.lower(), "v") for t in tokens]
with open("lemmatise.txt", "w") as output:
    output.write(str(tokens_norm))
print(f"Number of unique tokens: {len(set(tokens_norm))}")

# Task 4 -  Machine Learning

# a. Tfidf and K-means classifier analyses 

In [ ]:
# Import needed packages
import pandas as pd
import numpy as np
import nltk
ntlk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

# Prepare corpus for ML

In [ ]:
news =pd.read_csv('clean_body.csv')
news.head()

In [ ]:
# Split Corpus into train_test
train, test = train_test_split(news['clean_body'], test_size=0.2)
print(f"No. of training examples: {train.shape[0]}")
print(f"No. of testing examples: {test.shape[0]}")

In [ ]:
# Tokenise train data
train = train.to_string() 
tokeniser = RegexpTokenizer("[A-Za-z]+")
tokens_train = tokeniser.tokenize(train)
print(tokens[:20], "\n")

In [ ]:
# Lemmatise train data
lemmatiser = WordNetLemmatizer()
train_norm = [lemmatiser.lemmatize(t.lower(), "v") for t in tokens_train]
print(f"Number of unique tokens: {len(set(tokens_norm))}")

In [ ]:
# Tokenise test data
test = test.to_string() 
tokeniser = RegexpTokenizer("[A-Za-z]+")
tokens_test = tokeniser.tokenize(test)
print(tokens[:20], "\n")

In [ ]:
#  Lemmatise test data
lemmatiser = WordNetLemmatizer()
test_norm = [lemmatiser.lemmatize(t.lower(), "v") for t in tokens_test]
print(f"Number of unique tokens: {len(set(tokens_norm))}")

In [ ]:
# Tf-Idf Vectorisation
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(train_norm) 
X_test_vectors_tfidf = tfidf_vectorizer.transform(test_norm)

In [ ]:
print(X_train_vectors_tfidf)

In [ ]:
print(X_test_vectors_tfidf)

In [ ]:
# Scale the train data
scaler = StandardScaler(with_mean= False)
scaled_train = scaler.fit_transform(X_train_vectors_tfidf)
display(scaled_train)

In [ ]:
# Plot the sparse train matrix  # This was not useful so not included in report
# Get the non-zero indices and values of the sparse matrix
sparse_matrix=scaled_train
nonzero_indices = sparse_matrix.nonzero()
nonzero_values = sparse_matrix.data

# Create a scatter plot of the sparse matrix
plt.scatter(nonzero_indices[1], nonzero_indices[0], c=nonzero_values, cmap='YlGnBu')
plt.colorbar()
plt.title('Scaled Train Scatter Plot')
plt.show()

In [ ]:
# Scale the test data
scaler = StandardScaler(with_mean= False)
scaled_test = scaler.fit_transform(X_test_vectors_tfidf)
display(scaled_test)

In [ ]:
# Plot the sparse test matrix  # This was not useful so not included in report
# Get the non-zero indices and values of the sparse matrix
sparse_matrix=scaled_test
nonzero_indices = sparse_matrix.nonzero()
nonzero_values = sparse_matrix.data

# Create a scatter plot of the sparse matrix
plt.scatter(nonzero_indices[1], nonzero_indices[0], c=nonzero_values, cmap='YlGnBu')
plt.colorbar()
plt.title('Scaled Test Scatter Plot')
plt.show()

# Testing different cluster sizes to identify elbow for K-means

In [ ]:
sse = []
list_k = list(range(1, 4))

for k in list_k:
    km = KMeans(n_clusters=k, n_init=1)
    km.fit(scaled_train)
    sse.append(km.inertia_)

# Plot sse against k
plt.figure(figsize=(6, 6))
plt.plot(list_k, sse, '-o')
plt.xlabel(r'Number of clusters *k*')
plt.ylabel('Sum of squared distance');

In [ ]:
sse = []
list_k = list(range(5, 10))

for k in list_k:
    km = KMeans(n_clusters=k, n_init=1)
    km.fit(scaled_train)
    sse.append(km.inertia_)

# Plot sse against k
plt.figure(figsize=(6, 6))
plt.plot(list_k, sse, '-o')
plt.xlabel(r'Number of clusters *k*')
plt.ylabel('Sum of squared distance');

In [ ]:
sse = []
list_k = list(range(9, 15))

for k in list_k:
    km = KMeans(n_clusters=k, n_init=1)
    km.fit(scaled_train)
    sse.append(km.inertia_)

# Plot sse against k
plt.figure(figsize=(6, 6))
plt.plot(list_k, sse, '-o')
plt.xlabel(r'Number of clusters *k*')
plt.ylabel('Sum of squared distance');

In [ ]:
sse = []
list_k = list(range(16, 200))

for k in list_k:
    km = KMeans(n_clusters=k, n_init=1)
    km.fit(scaled_train)
    sse.append(km.inertia_)

# Plot sse against k
plt.figure(figsize=(6, 6))
plt.plot(list_k, sse, '-o')
plt.xlabel(r'Number of clusters *k*')
plt.ylabel('Sum of squared distance');

# Using PCA / SVD to reduce dimensions, before  k-means: 

In [ ]:
# https://medium.com/analytics-vidhya/implementation-of-principal-component-analysis-pca-in-k-means-clustering-b4bc0aa79cb6
wcss = []
for i in range(1,11):
   model = KMeans(n_clusters = i, init = "k-means++")
   model.fit(scaled_train)
   wcss.append(model.inertia_)
plt.figure(figsize=(10,10))
plt.plot(range(1,11), wcss)
plt.xlabel('Number of clusters')
plt.ylabel('WCSS') # sum of squares of distances of datapoints
plt.show()

In [ ]:
# PCA would not accept sparse matrix
pca = PCA(2)  
data = pca.fit_transform(scaled_train)

In [ ]:
# Try SVD to reduce dimensionality - rejected
# https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html

X = csr_matrix(scaled_train)
svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)
svd.fit(X)

print(svd.explained_variance_ratio_)  # paltry

In [ ]:
print(svd.explained_variance_ratio_.sum()) # paltry

In [ ]:
print(svd.singular_values_)

In [ ]:
#Plot variance explained
var_explained = svd.explained_variance_ratio_
var_explained
 
sns.barplot(x=list(range(1,len(var_explained)+1)),
            y=var_explained, color="teal")
plt.xlabel('SVs', fontsize=16)
plt.ylabel('Percent Variance Explained', fontsize=16)


# Plotting clusters against silhouette score- to determine optimum number of k-means clusters

In [ ]:
# A list holds the silhouette coefficients for each k
# 1 TO 10
silhouette_coefficients = []
for k in range(1, 10):
    kmeans = KMeans(init="random", n_clusters=k, n_init=10, max_iter=300,random_state=42)
    kmeans.fit(scaled_train)
    score = silhouette_score(scaled_train, kmeans.labels_)
    silhouette_coefficients.append(score)
    
plt.style.use("fivethirtyeight")
plt.plot(range(1, 11), silhouette_coefficients)
plt.xticks(range(1, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("Silhouette Coefficient")
plt.show()

In [ ]:
#  9 TO 20
silhouette_coefficients = []
for k in range(9, 20):
    kmeans = KMeans(init="random", n_clusters=k, n_init=10, max_iter=300,random_state=42)
    kmeans.fit(scaled_train)
    score = silhouette_score(scaled_train, kmeans.labels_)
    silhouette_coefficients.append(score)
    
plt.style.use("fivethirtyeight")
plt.plot(range(9, 20), silhouette_coefficients)
plt.xticks(range(9, 20))
plt.xlabel("Number of Clusters")
plt.ylabel("Silhouette Coefficient")
plt.show()

In [ ]:
# 19 TO 50
silhouette_coefficients = []
for k in range(19, 50):
    kmeans = KMeans(init="random", n_clusters=k, n_init=10, max_iter=300,random_state=42)
    kmeans.fit(scaled_train)
    score = silhouette_score(scaled_train, kmeans.labels_)
    silhouette_coefficients.append(score)
    
plt.style.use("fivethirtyeight")
plt.plot(range(19, 50), silhouette_coefficients)
plt.xticks(range(19, 50))
plt.xlabel("Number of Clusters")
plt.ylabel("Silhouette Coefficient")
plt.show()



In [ ]:
# THIS GRAPH WAS CHOSEN -63 CLUSTERS
# 49 TO 70
silhouette_coefficients = []
for k in range(49, 70):
    kmeans = KMeans(init="random", n_clusters=k, n_init=10, max_iter=300,random_state=42)
    kmeans.fit(scaled_train)
    score = silhouette_score(scaled_train, kmeans.labels_)
    silhouette_coefficients.append(score)
    
plt.style.use("fivethirtyeight")
plt.plot(range(49, 70), silhouette_coefficients)
plt.xticks(range(49, 70))
plt.xlabel("Number of Clusters")
plt.ylabel("Silhouette Coefficient")
plt.show()

In [ ]:
silhouette_coefficients = []
for k in range(69, 90):
    kmeans = KMeans(init="random", n_clusters=k, n_init=10, max_iter=300,random_state=42)
    kmeans.fit(scaled_train)
    score = silhouette_score(scaled_train, kmeans.labels_)
    silhouette_coefficients.append(score)
    
plt.style.use("fivethirtyeight")
plt.plot(range(69, 90), silhouette_coefficients)
plt.xticks(range(69, 90))
plt.xlabel("Number of Clusters")
plt.ylabel("Silhouette Coefficient")
plt.show()

In [ ]:
# 89 TO 100
silhouette_coefficients = []
for k in range(89, 100):
    kmeans = KMeans(init="random", n_clusters=k, n_init=10, max_iter=300,random_state=42)
    kmeans.fit(scaled_train)
    score = silhouette_score(scaled_train, kmeans.labels_)
    silhouette_coefficients.append(score)
    
plt.style.use("fivethirtyeight")
plt.plot(range(89, 100), silhouette_coefficients)
plt.xticks(range(89, 100))
plt.xlabel("Number of Clusters")
plt.ylabel("Silhouette Coefficient")
plt.show()

# Function for K-means fit and evaluation

In [ ]:
from collections import defaultdict
from time import time
from sklearn.cluster import KMeans
from sklearn import metrics

evaluations = []
evaluations_std = []


def fit_and_evaluate(km, X, name=None, n_runs=5):
    name = km.__class__.__name__ if name is None else name

    train_times = []
    scores = defaultdict(list)
    for seed in range(n_runs):
        km.set_params(random_state=seed)
        t0 = time()
        km.fit(X)
        # labels=km.labels_
        train_times.append(time() - t0)
        #labels = map(lambda x: dict(enumerate(X['target_names']))[x], X['target'])
        scores["Homogeneity"].append(metrics.homogeneity_score(labels, km.labels_))
        scores["Completeness"].append(metrics.completeness_score(labels, km.labels_))
        scores["V-measure"].append(metrics.v_measure_score(labels, km.labels_))
        scores["Adjusted Rand-Index"].append(
            metrics.adjusted_rand_score(labels, km.labels_)
        )
        scores["Silhouette Coefficient"].append(
            metrics.silhouette_score(X, km.labels_, sample_size=2000)
        )
    train_times = np.asarray(train_times)

    print(f"clustering done in {train_times.mean():.2f} � {train_times.std():.2f} s ")
    evaluation = {
        "estimator": name,
        "train_time": train_times.mean(),
    }
    evaluation_std = {
        "estimator": name,
        "train_time": train_times.std(),
    }
    for score_name, score_values in scores.items():
        mean_score, std_score = np.mean(score_values), np.std(score_values)
        print(f"{score_name}: {mean_score:.3f} � {std_score:.3f}")
        evaluation[score_name] = mean_score
        evaluation_std[score_name] = std_score
    evaluations.append(evaluation)
    evaluations_std.append(evaluation_std)

# Fitting k-means to scaled-train corpus

In [ ]:
kmTR = KMeans(
    n_clusters=63,
    max_iter=500,
    n_init=1,
)
labels=kmTR.fit_predict(scaled_train)
fit_and_evaluate(kmTR, scaled_train, name="KMeans",  n_runs=5)

In [ ]:
# plotting clusters for scaled train
# https://stackoverflow.com/questions/50898910/visualize-sparse-input-from-sklearn-kmeans-with-matplotlib

kmTR = KMeans(n_clusters = 63, init = 'k-means++', max_iter = 500, n_init = 1, verbose = False )
kmTR.fit(scaled_train)

plt.scatter(scaled_train[:, 0].toarray(), scaled_train[:, 1].toarray())
centroids = kmTR.cluster_centers_
plt.scatter(centroids[:, 0], centroids[:, 1], c='red', s=500, alpha=0.5)
plt.title('Visualisation of clusters (scaled train data)', fontweight='bold')
plt.show()

# Fitting k-means to scaled-test corpus

In [ ]:
kmTE = KMeans(
    n_clusters=63,
    max_iter=500,
    n_init=1,
)
labels=kmTE.fit_predict(scaled_test)
fit_and_evaluate(kmTE, scaled_test, name="KMeans",  n_runs=5)

In [ ]:
# plotting clusters for scaled test
# https://stackoverflow.com/questions/50898910/visualize-sparse-input-from-sklearn-kmeans-with-matplotlib 

kmTE = KMeans(n_clusters = 63, init = 'k-means++', max_iter = 500, n_init = 1, verbose = False )
kmTE.fit(scaled_test)

plt.scatter(scaled_test[:, 0].toarray(), scaled_test[:, 1].toarray())
centroids = kmTE.cluster_centers_
plt.scatter(centroids[:, 0], centroids[:, 1], c='red', s=500, alpha=0.5)
plt.title('Visualisation of clusters (scaled test data)', fontweight='bold')
plt.show()

# b. Sentiment and LDA Analyses


In [ ]:
import nltk
import pandas as pd
import matplotlib.pyplot as plt
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

In [ ]:
#Read in Data
news_df=pd.read_csv('clean_body_2.csv', names=['ID', 'News'])
news_df.head()

In [ ]:
# https://www.analyticsvidhya.com/blog/2021/06/vader-for-sentiment-analysis/
# Make new data frame with 'id' and 'news' fields
df_subset = news_df[['ID', 'News']].copy()
#data clean-up
#remove all non-aphabet characters
df_subset['News'] = df_subset['News'].str.replace("[^a-zA-Z#]", " ")
#convert to lower-case
df_subset['News'] = df_subset['News'].str.casefold()
print (df_subset.head(10))

In [ ]:
df_subset = df_subset.dropna()
print (df_subset.head(10))

In [ ]:
df_subset.drop([1], axis=0, inplace=True)
print (df_subset.head(10))

# Sentiment analysis with nltk-VADER

In [ ]:
# Set up empty dataframe for staging output
df1=pd.DataFrame()
df1['ID']=['99999999999']
df1['sentiment_type']='NA999NA'
df1['sentiment_score']=0

In [ ]:
# Set up for sentiment analysis
print('Processing sentiment analysis...')
sid = SentimentIntensityAnalyzer()
t_df = df1 
for index, row in df_subset.iterrows():
    scores = sid.polarity_scores(row[1])
    for key, value in scores.items():
        temp = [key,value,row[0]]
        df1['ID']=row[0]
        df1['sentiment_type']=key
        df1['sentiment_score']=value
        t_df=t_df._append(df1)
#remove dummy row with row_id = 99999999999
t_df_cleaned = t_df[t_df.ID != '99999999999']
#remove duplicates if any exist
t_df_cleaned = t_df_cleaned.drop_duplicates()
# only keep rows where sentiment_type = compound
t_df_cleaned = t_df[t_df.sentiment_type == 'compound']
print(t_df_cleaned.head(10))

In [ ]:
# Merge dataframes
df_output = pd.merge(news_df, t_df_cleaned, on='ID', how='inner')
print(df_output.head(10))

In [ ]:
df_output.to_csv('sentiment_scores.csv')

In [ ]:
df_output[["sentiment_score"]].describe()

In [ ]:
# Generate mean of sentiment_score by ID
dfg = df_output.groupby(['ID'])['sentiment_score'].mean()
#create a bar plot
dfg.plot(kind='bar', title='Sentiment Score', ylabel='Mean Sentiment Score',  xlabel=None, figsize=(6, 5))

# LDA sentiment analyses

In [ ]:
# Load needed packages
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import re
from pprint import pprint
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

In [ ]:
news_data = pd.read_csv('sentiment_scores.csv')
dislay(news_data)

In [ ]:
news_2_data=news_data[['News', 'sentiment_score']]

In [ ]:
# Generate positive and negative data files
pos_news = news_2_data[news_2_data['sentiment_score']>0.2]
neg_news = news_2_data[news_2_data['sentiment_score']<-0.2]
pos_news.head()


In [ ]:
pos_news.describe()

In [ ]:
neg_news.describe()

# Positive texts

In [ ]:
# Remove punctuation with re for positive news
pos_news['News_Processed'] = pos_news['News'].map(lambda x: re.sub('[,\.!?]', '', str(x)))
# Convert lowercase
pos_news['News_Processed'] = pos_news['News'].map(lambda x: x.lower())
pos_news['News_Processed'].head()

In [ ]:
# Generate Positive Wordcloud 
from wordcloud import WordCloud
long_string = ','.join(list(pos_news['News_Processed'].values))
# Create a WordCloud object
wordcloud = WordCloud(background_color="white", max_words=5000, contour_width=3, contour_color='steelblue')
# Generate a word cloud
wordcloud.generate(long_string)
# Visualize the word cloud
wordcloud.to_image()

In [ ]:
# Remove stopwords
stop_words = stopwords.words('english')
stop_words.extend(['nan', 'yes', 'no'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = pos_news.News_Processed.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

In [ ]:
# Create Dictionary
import gensim.corpora as corpora

id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

# LDA Trialling number of topics (coherence score)

In [ ]:
# LDA model trialling # number of topics (positive only)
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 20 topics
#pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
# Calculating perplexity and coherence score
from gensim.models import CoherenceModel
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
# Try 20 topics and check coherence..
num_topics = 20
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 20 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
# Try 50 topics and check coherence..
# number of topics
num_topics = 50
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 20 topics
#pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
# Try 5 topics and check coherence..
# number of topics
num_topics = 5
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 20 topics
#pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

# Chose 3 Topics

In [ ]:
# Final - 3 topics
from pprint import pprint
# number of topics
num_topics = 3
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 20 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
# Calculating perplexity and coherence score
from gensim.models import CoherenceModel
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
# Visualize the positive topics
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

   # Negative Texts

In [ ]:
# Remove punctuation with re
neg_news['News_Processed'] = neg_news['News'].map(lambda x: re.sub('[,\.!?]', '', str(x)))
# Convert lowercase
neg_news['News_Processed'] = neg_news['News_Processed'].map(lambda x: x.lower())
neg_news['News_Processed'].head()

In [ ]:
# Generate negative Wordcloud
long_string2 = ','.join(list(neg_news['News_Processed'].values))
long_string2 = long_string2.replace("nan","")
# Create a WordCloud object
wordcloud = WordCloud(background_color="white", max_words=5000, contour_width=3, contour_color='steelblue')
# Generate a word cloud
wordcloud.generate(long_string2)
# Visualize the word cloud
wordcloud.to_image()

In [ ]:
# Remove stopwords
data = neg_news.News_Processed.values.tolist()
data_words2 = list(sent_to_words(data))
# remove stop words
data_words2 = remove_stopwords(data_words2)
print(data_words2[:10][9][:30])

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_words2)
# Create Corpus
texts = data_words2
# Term Document Frequency
corpus2 = [id2word.doc2bow(text) for text in texts]
# View
print(corpus2[:10][9][:30])

In [ ]:
# number of topics = 3
num_topics = 3
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus2,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 5 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus2]

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus2))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
# Visualize the negative topics
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus2, dictionary=lda_model.id2word)
vis